In [31]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from utils.labels import trainId2label
import os
import cv2


In [ ]:
# Path to Cityscapes dataset
CITYSCAPES_DIR = "data"
IMAGES_DIR = os.path.join(CITYSCAPES_DIR, "leftImg8bit_trainvaltest/leftImg8bit/train")
LABELS_DIR = os.path.join(CITYSCAPES_DIR, "gtFine_trainvaltest/gtFine/train")
# Limit the number of images to process
LIMIT = 10


In [33]:
def get_city_folders(directory):
    """Retrieve all city subfolders."""
    return [os.path.join(directory, city) for city in os.listdir(directory) if os.path.isdir(os.path.join(directory, city))]


In [34]:
def get_image_label_pairs(images_dir, labels_dir):
    """Match RGB images with their corresponding label IDs."""
    image_label_pairs = []
    for city_folder in get_city_folders(images_dir):
        city_name = os.path.basename(city_folder)
        label_city_folder = os.path.join(labels_dir, city_name)

        for image_file in os.listdir(city_folder):
            if image_file.endswith("_leftImg8bit.png"):
                # Image path
                image_path = os.path.join(city_folder, image_file)
                # Corresponding label file
                label_file = image_file.replace("_leftImg8bit.png", "_gtFine_labelIds.png")
                label_path = os.path.join(label_city_folder, label_file)
                if os.path.exists(label_path):
                    image_label_pairs.append((image_path, label_path))
                    if len(image_label_pairs) >= LIMIT:  # Stop if limit is reached
                        return image_label_pairs
    return image_label_pairs


In [35]:
def preprocess_data(image_label_pairs):
    """Extract pixel RGB values and labels for training."""
    X = []  # Pixel RGB values
    y = []  # Binary labels (road or not road)

    for image_path, label_path in image_label_pairs:
        # Load RGB image
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        # Load label image
        label_img = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)
        
        # Flatten image and labels into 1D arrays
        image_pixels = image.reshape(-1, 3)  # RGB values
        label_pixels = label_img.flatten()  # Label IDs
        
        # Filter for road (TrainId = 7) and not-road
        road_mask = (label_pixels == 7)
        not_road_mask = ~road_mask
        
        # Append data
        X.extend(image_pixels[road_mask])  # Road pixels
        y.extend([1] * np.sum(road_mask))  # Label as road
        
        X.extend(image_pixels[not_road_mask])  # Non-road pixels
        y.extend([0] * np.sum(not_road_mask))  # Label as not-road

    return np.array(X), np.array(y)

In [ ]:
# Get image-label pairs
image_label_pairs = get_image_label_pairs(IMAGES_DIR, LABELS_DIR)

# Preprocess data
X, y = preprocess_data(image_label_pairs)

In [1]:
print(f"Processed {len(X)} pixels")

NameError: name 'X' is not defined

In [ ]:
# Normalize RGB values
X = X / 255.0  # Scale RGB to [0, 1]

In [ ]:
# Model training pipeline
import tensorflow as tf
from tensorflow.keras import layers, models

In [6]:
# Generate dataset
def load_cityscapes_dataset(images_dir, labels_dir):
    images, labels = [], []

    for city in os.listdir(images_dir):
        city_img_dir = os.path.join(images_dir, city)
        city_label_dir = os.path.join(labels_dir, city)
        
        for img_file in os.listdir(city_img_dir):
            if not img_file.endswith("_leftImg8bit.png"):
                continue
            
            # File paths
            image_path = os.path.join(city_img_dir, img_file)
            label_path = os.path.join(city_label_dir, img_file.replace("_leftImg8bit.png", "_gtFine_labelIds.png"))
            
            # Load and preprocess
            image, label = load_image_and_label(image_path, label_path)
            images.append(image)
            labels.append(label)
    
    return np.array(images), np.array(labels)

In [7]:
# Create dataset
print("Loading Cityscapes data...")
X, Y = load_cityscapes_dataset(IMAGES_DIR, LABELS_DIR)

# Flatten pixels for input-output training
X_flat = X.reshape(-1, 3)  # RGB for each pixel
Y_flat = Y.flatten()  # Corresponding binary label


Loading Cityscapes data...


FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: '/path/to/cityscapes\\leftImg8bit/train'

In [9]:
# Define model
print("Building model...")
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3,)),  # RGB input
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Building model...


In [10]:
# Train model
print("Training model...")
model.fit(X_flat, Y_flat, batch_size=128, epochs=5, validation_split=0.1)

Training model...


NameError: name 'X_flat' is not defined

In [11]:
# Save model
model.save("pixel_classifier.h5")
print("Model saved as pixel_classifier.h5")

Model saved as pixel_classifier.h5


In [ ]:
# Test example
def predict_pixel_class(rgb):
    prediction = model.predict(np.array([rgb]))
    return 1 if prediction > 0.5 else 0

print("Sample prediction:", predict_pixel_class([128, 64, 128]))  # Road-like color